# R-Tree Regions
This notebook steps through using python's rtree package, which leverages libspatialindex, for identifying regions with uniform spatial load.
## Basics

In [1]:
# Import and print version.  Notebook created with version 1.8.5
from rtree import index
index.__c_api_version__.decode('utf-8')

'1.8.5'

In [2]:
# Construct an instance of the r-tree
idx = index.Index()

In [3]:
# Create a bounding box.  Shows how the corners are stored by default in the Index (not interleaved)
left, bottom, right, top = (0.0, 0.0, 1.0, 1.0)

In [4]:
# Inserting ID, (BBox)
idx.insert(0, (left, bottom, right, top))

## Query the Index

In [5]:
# Intersection
list(idx.intersection((1.0, 1.0, 2.0, 2.0)))

[0]

In [6]:
list(idx.intersection((1.0000001, 1.0000001, 2.0, 2.0)))

[]

In [7]:
# Nearest Neighbors
idx.insert(1, (left, bottom, right, top))
list(idx.nearest((1.0000001, 1.0000001, 2.0, 2.0)))

[0, 1]

## Object Database
Additional information can be stored with the bounding boxes in the tree.  This will be useful for storing relevant information about the bounding box.

In [8]:
# Any serializable object canbe added to the index (Clustered index in libspatialindex)
idx.insert(id=2, coordinates=(left, bottom, right, top), obj=42)

In [9]:
[n.object for n in idx.intersection((left, bottom, right, top), objects=True)]

[None, None, 42]

## Saving the tree
The r-tree can be saved and loaded up later.  This is useful for maintaining the tree should new objects come into play.  It's also useful should the server need restarted since the tree doesn't need to be recalculated.

In [10]:
# File naming is controled using the properties.
p = index.Property()
p.dat_extension = 'data'
p.idx_extension = 'index'

# Create an index file called 'rtree'.  If 'rtree' already exists it will be opened for appending.
file_idx = index.Rtree('rtree', properties=p)
file_idx.insert(1, (left, bottom, right, top))
file_idx.insert(2, (left-1.0, bottom-1.0, right+1.0, top+1.0))
[n for n in file_idx.intersection((left, bottom, right, top))]

[1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]

In [11]:
# Delete the existing tree and then re-open.  Query for the items inserted above.
del(file_idx)
file_idx = index.Rtree('rtree', properties=p)
[n for n in file_idx.intersection((left, bottom, right, top))]

[1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]

## Adjusting the number of boxes in the leaf
For my application I need to adjust the number of items that will be in each calculated bounding box.  The example below shows how this can be done.

In [12]:
from json import dumps
from itertools import islice

leaf_size = 100  # Minimum size is 4, default is 100

# Setup the properties for creating the index
p = index.Property()
p.leaf_capacity = leaf_size
p.near_minimum_overlap_factor = leaf_size
p.index_capacity = leaf_size
idx = index.Index(properties=p)
idx.properties

{'pagesize': 4096, 'point_pool_capacity': 500, 'type': 0, 'variant': 2, 'fill_factor': 0.7, 'writethrough': False, 'custom_storage_callbacks_size': 0, 'tight_mbr': True, 'storage': 0, 'index_id': None, 'index_capacity': 100, 'region_pool_capacity': 1000, 'filename': '', 'dat_extension': 'dat', 'idx_extension': 'idx', 'split_distribution_factor': 0.4, 'reinsert_factor': 0.3, 'tpr_horizon': 20.0, 'dimension': 2, 'near_minimum_overlap_factor': 100, 'leaf_capacity': 100, 'custom_storage_callbacks': None, 'overwrite': True, 'buffering_capacity': 10}

## Query the TrajCube Pre-aggregations
This portion steps through querying the spatial objects relevant to the tree and then finding the optimal spatial regions to be used for spatial querying in the final solution.

In [13]:
# Import django ORM objects
from streetcube.models import StreetTaxiCell
from osm.models import PlanetOsmLine

# Get a test object.
cell = StreetTaxiCell.objects.all()[2]
cell.osm

<PlanetOsmLine: undefined>

In [14]:
# Query for all streets in the main cube
from datetime import datetime
st = datetime.now()
qset = StreetTaxiCell.objects.distinct('osm__gid')
qset_t = datetime.now()
qset_len = len(qset)
len_t = datetime.now()
print('Count', 'QuerySet creation time', 'QuerySet Execution')
qset_len-1, str(qset_t - st), str(len_t - qset_t)

Count QuerySet creation time QuerySet Execution


(5628, '0:00:00.000255', '0:02:29.239952')

In [15]:
# Quickly list a few items just to show how bounding boxes are determined.
from itertools import islice
from json import loads
for cell in islice(qset, 0, 10):
    test = cell.osm
    print(cell.osm.way.extent, cell.osm)

(13377387.28, 3537070.13, 13377452.91, 3538311.54) XinhuaRoad
(13376299.72, 3539414.03, 13378159.56, 3539897.2) ChaohuiRoad
(13372308.22, 3536375.65, 13375073.56, 3538707.53) ShuguangRoad
(13374855.08, 3543796.93, 13375116.66, 3543973.91) DaguanRoad
(13377504.93, 3539466.9, 13377680.73, 3540197.56) NorthJianguoRoad
(13371642.56, 3538765.39, 13375006.03, 3538964.75) TianMuShanRoad
(13376346.31, 3539085.2, 13378510.18, 3539314.03) HuanChengNorthRoad
(13375073.56, 3538553.16, 13378201.19, 3538711.45) TiyuchangRoad
(13372248.68, 3544008.55, 13372494.81, 3544336.86) GucuiRoad
(13373625.93, 3539571.09, 13374645.85, 3539635.33) WensanRoad


## Add streets to the R-Tree

In [16]:
from itertools import islice
from rtree import index

leaf_size = 20  # Minimum allowed size is 4

# Setup the properties for creating the index
p = index.Property()
p.leaf_capacity = leaf_size
p.near_minimum_overlap_factor = leaf_size
p.index_capacity = leaf_size
idx = index.Index(properties=p)
idx.interleaved = True  # Interleaved to align with django GEOS API's extent method (xmin, ymin, xmax, ymax)

# Add test data to the index
for i, cell in enumerate(islice(qset, 0, None)):
    # Coordinates are transformed here because they are easier to plot with the current tool chain.
    try:
        geom = cell.osm.way.transform(4326, clone=True)
        idx.insert(cell.osm.gid, geom.extent, obj=loads(geom.json))
    except AttributeError:
        pass  # There will be one cell representing an total aggregation where 'osm' is None

In [17]:
# Show how the tree size is determined.  Should match (len(qset)-1) for the number of streets above.
idx.count(idx.bounds)

5628

## GeoJSON Output

In [18]:
# Generate GeoJSON output of the bounding box of each leaf container in the r-tree.
# The bounds provided in the objects returned from leaves seem to be minimal bounds related to contained objects
from json import dumps
from collections import namedtuple
collection = {
    'type': 'FeatureCollection',
    'features': []
}
Leaf = namedtuple('Leaf', ['id', 'child_ids', 'bbox'])
total_count = 0
for l in map(Leaf._make, islice(idx.leaves(), 0, None)):
    total_count += (len(l.child_ids))
    #print(len(l.child_ids))
    collection['features'].append({
            'type': 'Feature',
            'geometry': {
                'type': 'Polygon',
                'coordinates': [[
                    [l.bbox[0], l.bbox[1]],
                    [l.bbox[0], l.bbox[3]],
                    [l.bbox[2], l.bbox[3]],
                    [l.bbox[2], l.bbox[1]],
                    [l.bbox[0], l.bbox[1]],
                ]]
            },
            'properties': {
                'id': l.id
            }
        })
# print(total_count)
dumps(collection)

'{"features": [{"type": "Feature", "properties": {"id": 381}, "geometry": {"type": "Polygon", "coordinates": [[[119.93902653394778, 30.193897988032262], [119.93902653394778, 30.226483015573045], [120.02176568352857, 30.226483015573045], [120.02176568352857, 30.193897988032262], [119.93902653394778, 30.193897988032262]]]}}, {"type": "Feature", "properties": {"id": 114}, "geometry": {"type": "Polygon", "coordinates": [[[119.94727163095156, 30.148236267177314], [119.94727163095156, 30.195715382926736], [120.03784992885919, 30.195715382926736], [120.03784992885919, 30.148236267177314], [119.94727163095156, 30.148236267177314]]]}}, {"type": "Feature", "properties": {"id": 117}, "geometry": {"type": "Polygon", "coordinates": [[[120.00294876310366, 30.2100639144474], [120.00294876310366, 30.24812129573761], [120.04870166732287, 30.24812129573761], [120.04870166732287, 30.2100639144474], [120.00294876310366, 30.2100639144474]]]}}, {"type": "Feature", "properties": {"id": 324}, "geometry": {"ty

## Getting the spatially associated objects
The method index.leaves() will return a list of (rtree-id, child-objects-ids, min-bounds).  That can be used to find the set of bounding boxes defining regions for use in the queries.  This is used in conjunction with the following GeoJSON outputs to show details of the tree.  Output can be pasted into http://geojson.io to visualize.
### GeoJSON of the container bounding boxes.

In [19]:
# Generate GeoJSON output of a single leaf in the r-tree but include the children bbox and/or road segment.
from json import dumps
from collections import namedtuple
collection = {
    'type': 'FeatureCollection',
    'features': []
}
Leaf = namedtuple('Leaf', ['id', 'child_ids', 'bbox'])
for l in map(Leaf._make, islice(idx.leaves(), 0, None)):
    collection['features'].append({
            'type': 'Feature',
            'geometry': {
                'type': 'Polygon',
                'coordinates': [[
                    [l.bbox[0], l.bbox[1]],
                    [l.bbox[0], l.bbox[3]],
                    [l.bbox[2], l.bbox[3]],
                    [l.bbox[2], l.bbox[1]],
                    [l.bbox[0], l.bbox[1]],
                ]]
            },
            'properties': {
                'id': l.id,
                'parent': True
            }
        })
        
dumps(collection)

'{"features": [{"type": "Feature", "properties": {"id": 381, "parent": true}, "geometry": {"type": "Polygon", "coordinates": [[[119.93902653394778, 30.193897988032262], [119.93902653394778, 30.226483015573045], [120.02176568352857, 30.226483015573045], [120.02176568352857, 30.193897988032262], [119.93902653394778, 30.193897988032262]]]}}, {"type": "Feature", "properties": {"id": 114, "parent": true}, "geometry": {"type": "Polygon", "coordinates": [[[119.94727163095156, 30.148236267177314], [119.94727163095156, 30.195715382926736], [120.03784992885919, 30.195715382926736], [120.03784992885919, 30.148236267177314], [119.94727163095156, 30.148236267177314]]]}}, {"type": "Feature", "properties": {"id": 117, "parent": true}, "geometry": {"type": "Polygon", "coordinates": [[[120.00294876310366, 30.2100639144474], [120.00294876310366, 30.24812129573761], [120.04870166732287, 30.24812129573761], [120.04870166732287, 30.2100639144474], [120.00294876310366, 30.2100639144474]]]}}, {"type": "Featu

### GeoJSON of one container with content details

In [20]:
# Generate GeoJSON output of a single leaf in the r-tree but include the children bbox and/or road segment.
from json import dumps
from collections import namedtuple
collection = {
    'type': 'FeatureCollection',
    'features': []
}
Leaf = namedtuple('Leaf', ['id', 'child_ids', 'bbox'])
for l in map(Leaf._make, islice(idx.leaves(), 0, None)):
    collection['features'].append({
            'type': 'Feature',
            'geometry': {
                'type': 'Polygon',
                'coordinates': [[
                    [l.bbox[0], l.bbox[1]],
                    [l.bbox[0], l.bbox[3]],
                    [l.bbox[2], l.bbox[3]],
                    [l.bbox[2], l.bbox[1]],
                    [l.bbox[0], l.bbox[1]],
                ]]
            },
            'properties': {
                'id': l.id,
                'parent': True
            }
        })
    for i in islice(idx.intersection(l.bbox, objects=True), 0, None):
        collection['features'].append({
                'type': 'Feature',
                'geometry': {
                    'type': 'Polygon',
                    'coordinates': [[
                        [i.bbox[0], i.bbox[1]],
                        [i.bbox[0], i.bbox[3]],
                        [i.bbox[2], i.bbox[3]],
                        [i.bbox[2], i.bbox[1]],
                        [i.bbox[0], i.bbox[1]],
                    ]]
                },
                'properties': {
                    'id': i.id,
                    'child': True
                }
            })
        collection['features'].append({
                'type': 'Feature',
                'geometry': i.object,
                'properties': {
                    'bbox_id': i.id
                }
            })
    break
        
dumps(collection)

'{"features": [{"type": "Feature", "properties": {"id": 381, "parent": true}, "geometry": {"type": "Polygon", "coordinates": [[[119.93902653394778, 30.193897988032262], [119.93902653394778, 30.226483015573045], [120.02176568352857, 30.226483015573045], [120.02176568352857, 30.193897988032262], [119.93902653394778, 30.193897988032262]]]}}, {"type": "Feature", "properties": {"id": 1764208, "child": true}, "geometry": {"type": "Polygon", "coordinates": [[[119.97877932088981, 30.226483015573045], [119.97877932088981, 30.235229114673523], [119.97918580855587, 30.235229114673523], [119.97918580855587, 30.226483015573045], [119.97877932088981, 30.226483015573045]]]}}, {"type": "Feature", "properties": {"bbox_id": 1764208}, "geometry": {"type": "LineString", "coordinates": [[119.97918580855587, 30.235229114673523], [119.97915472684704, 30.234799069365202], [119.97898251980709, 30.23095776006738], [119.9788203738983, 30.22738827284575], [119.97877932088981, 30.226483015573045]]}}, {"type": "Fea

### One container with detail using maximum bounds

In [21]:
# Generate GeoJSON output of a single leaf in the r-tree but include the children bbox and/or road segment.
from json import dumps
from collections import namedtuple
collection = {
    'type': 'FeatureCollection',
    'features': []
}
Leaf = namedtuple('Leaf', ['id', 'child_ids', 'bbox'])
for l in map(Leaf._make, islice(idx.leaves(), 0, None)):
    for i in islice(idx.intersection(l.bbox, objects=True), 0, None):
        collection['features'].append({
                'type': 'Feature',
                'geometry': {
                    'type': 'Polygon',
                    'coordinates': [[
                        [i.bbox[0], i.bbox[1]],
                        [i.bbox[0], i.bbox[3]],
                        [i.bbox[2], i.bbox[3]],
                        [i.bbox[2], i.bbox[1]],
                        [i.bbox[0], i.bbox[1]],
                    ]]
                },
                'properties': {
                    'id': i.id,
                    'child': True
                }
            })
        collection['features'].append({
                'type': 'Feature',
                'geometry': i.object,
                'properties': {
                    'bbox_id': i.id
                }
            })
        if i.bbox[0] < l.bbox[0]:
            l.bbox[0] = i.bbox[0]
        if i.bbox[1] < l.bbox[1]:
            l.bbox[1] = i.bbox[1]
        if i.bbox[2] > l.bbox[2]:
            l.bbox[2] = i.bbox[2]
        if i.bbox[3] > l.bbox[3]:
            l.bbox[3] = i.bbox[3]
    
    collection['features'].append({
            'type': 'Feature',
            'geometry': {
                'type': 'Polygon',
                'coordinates': [[
                    [l.bbox[0], l.bbox[1]],
                    [l.bbox[0], l.bbox[3]],
                    [l.bbox[2], l.bbox[3]],
                    [l.bbox[2], l.bbox[1]],
                    [l.bbox[0], l.bbox[1]],
                ]]
            },
            'properties': {
                'id': l.id,
                'parent': True
            }
        })
    
    break
        
dumps(collection)

'{"features": [{"type": "Feature", "properties": {"id": 1764208, "child": true}, "geometry": {"type": "Polygon", "coordinates": [[[119.97877932088981, 30.226483015573045], [119.97877932088981, 30.235229114673523], [119.97918580855587, 30.235229114673523], [119.97918580855587, 30.226483015573045], [119.97877932088981, 30.226483015573045]]]}}, {"type": "Feature", "properties": {"bbox_id": 1764208}, "geometry": {"type": "LineString", "coordinates": [[119.97918580855587, 30.235229114673523], [119.97915472684704, 30.234799069365202], [119.97898251980709, 30.23095776006738], [119.9788203738983, 30.22738827284575], [119.97877932088981, 30.226483015573045]]}}, {"type": "Feature", "properties": {"id": 1764296, "child": true}, "geometry": {"type": "Polygon", "coordinates": [[[119.95847272422925, 30.224568698486237], [119.95847272422925, 30.234870704908094], [119.9597695321734, 30.234870704908094], [119.9597695321734, 30.224568698486237], [119.95847272422925, 30.224568698486237]]]}}, {"type": "Fe

## Adding regions to the database
Cells in this section of the notebook are design to run independent of other sections in the notebook.

In [22]:
# Used to expand the bbox to the full extent of its contained features.
def expand_bbox(bbox, features):
    for i in features:
        if i.bbox[0] < bbox[0]:
            bbox[0] = i.bbox[0]
        if i.bbox[1] < bbox[1]:
            bbox[1] = i.bbox[1]
        if i.bbox[2] > bbox[2]:
            bbox[2] = i.bbox[2]
        if i.bbox[3] > bbox[3]:
            bbox[3] = i.bbox[3]

In [23]:
# NOTE: This cell is designed to run independent of the other cells in the notebook.
from json import loads
from rtree import index
from collections import namedtuple

from streetcube.models import RegionPartition
from django.contrib.gis.geos import GEOSGeometry


def osm_line_generator():
    print('Getting query set')
    qset = StreetTaxiCell.objects.distinct('osm__gid')
    for cell in qset:
        try:
            yield (cell.osm.gid, cell.osm.way.extent, loads(cell.osm.way.json))
        except AttributeError:
            pass  # There will be one cell representing an total aggregation where 'osm' is None


# Setup the properties for creating the index
leaf_size = 200  # Minimum allowed size is 4
p = index.Property()
p.leaf_capacity = leaf_size
p.index_capacity = leaf_size
p.buffering_capacity = 10000
p.storage = index.RT_Memory
# print(p)
idx = index.Index(osm_line_generator(), interleaved=True, properties=p)
# print(idx.interleaved)


Leaf = namedtuple('Leaf', ['id', 'child_ids', 'bbox'])

del_items = RegionPartition.objects.filter(name__isnull=True).delete()
print('Removed {} previous regions'.format(del_items))

for l in map(Leaf._make, idx.leaves()):
    # # Expand to complete bounding box.
    # expand_bbox(l.bbox, idx.intersection(l.bbox, objects=True))
    region = RegionPartition()
    region.geometry = GEOSGeometry('SRID=3857;POLYGON (({xmin} {ymin}, {xmin} {ymax}, {xmax} {ymax}, {xmax} {ymin}, {xmin} {ymin}))'.format(
        xmin=l.bbox[0], ymin=l.bbox[1],
        xmax=l.bbox[2], ymax=l.bbox[3]
    ))
    region.items = l.child_ids
    region.save()
'Added {} regions'.format(len(idx.leaves()))

Getting query set
Removed (41, {'streetcube.RegionPartition': 41}) previous regions


'Added 41 regions'

In [24]:
region = RegionPartition.objects.filter(name__isnull=True)[0]
region

<RegionPartition: RegionPartition object>

In [25]:
region.geometry.json

'{"type": "Polygon", "coordinates": [[[13320215.99, 3551414.51], [13320215.99, 3585630.65], [13376339.6, 3585630.65], [13376339.6, 3551414.51], [13320215.99, 3551414.51]]]}'